## Reference

Custom Dataset classes in pytorch
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

k-Fold validation pytorch
--
1. https://stackoverflow.com/questions/58996242/cross-validation-for-mnist-dataset-with-pytorch-and-sklearn
2. https://discuss.pytorch.org/t/i-need-help-in-this-k-fold-cross-validation-implementation/90705/5
3. https://github.com/buomsoo-kim/PyTorch-learners-tutorial/blob/master/PyTorch%20Basics/pytorch-datasets-2.ipynb


kFold split sklearn
--
1. sklearn.model_selection.KFold -  normal ordered splits without any shuffle by default. 
2. sklearn.model_selection.StratifiedKFold - tries to preserve the distribution of each class in each set
3. GroupKFold - ensures the group of data is not repeated in any fold; little complex concept
4. RepeatedKFold - repeat kfold n times with different random state each instance


Pytorch Lightning 
--
1. https://www.kaggle.com/pytorchlightning/pytorch-on-tpu-with-pytorch-lightning
2. https://www.kaggle.com/arroqc/siim-isic-pytorch-lightning-starter-seresnext50/notebook

In [1]:
#!pip install -U skorch

## Library imports

In [2]:
# common imports
import os
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import math
#import time
#from skimage import io, transform
#from typing import Dict
#from pathlib import Path

# interactive plot libraries
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot # download_plotlyjs, plot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)


# torch imports
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.nn.functional as F


# sklearn related imports
# import skorch #sklearn + pytorch functionalitites
from sklearn.model_selection import StratifiedKFold #KFold, 
#from sklearn.model_selection import cross_val_score

#import skorch
#from skorch.callbacks import Checkpoint
#from skorch.callbacks import Freezer
#from skorch.helper import predefined_split
#from skorch import NeuralNetClassifier

# lightning imports
import pytorch_lightning as pl


## Config files

In [3]:
path_cfg = {'train_img_path': "cassava-leaf-disease-classification/train_images/",
            'train_csv_path': 'cassava-leaf-disease-classification/train.csv',
            'train' : True, 'lr_find' : False, 'validate' : False, 'test' : False}

model_cfg = {'model_architecture': 'resnet18', 'model_name': 'R18_imagenet_v1',
             'init_lr': 4e-4, 'weight_path': '', 'train_epochs':3}

train_cfg = {'batch_size': 16, 'shuffle': False, 'num_workers': 4, 'checkpt_every' : 1 }
valid_cfg = {'batch_size': 16, 'shuffle': False, 'num_workers': 4, 'validate_every' : 1 }
test_cfg  = {'batch_size': 16, 'shuffle': False, 'num_workers': 4}

In [4]:
index_label_map = {
                0: "Cassava Bacterial Blight (CBB)", 
                1: "Cassava Brown Streak Disease (CBSD)",
                2: "Cassava Green Mottle (CGM)", 
                3: "Cassava Mosaic Disease (CMD)", 
                4: "Healthy"
                }

## TODO

- load images into dataset (Dataset class of pytorch maybe)
- split into 5 fold data - scikit learn
- simple network -r18, r50 with last layers changed to 5 lables
- adam optimizer, lr_finder, cross entropy loss
- cv score

## Helper functions

In [5]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

In [6]:
def get_cv_splits(csv_path, cv_splits=3):
    df = pd.read_csv(csv_path)
    y = df['label'].values
    X = np.zeros(y.shape)
    
    cv_split_fn = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=RANDOM_STATE)
    
    cv_split_idx = {}
    for idx, (train_idx, test_idx) in enumerate(cv_split_fn.split(X,y)):
        cv_split_idx['split' + str(idx+1) + '_train'] = train_idx
        cv_split_idx['split' + str(idx+1) + '_test']  = test_idx
    return cv_split_idx

In [7]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

## Dataset class

In [8]:
class CassavaDataset(Dataset):
    """Cassave leaf disease detection dataset."""

    def __init__(self, csv_file, root_dir, transform=None, idx_list=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            idx_list (list of ints): select only certain rows from csv 
        """
        self.cassava_leaf_disease = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        if idx_list != None:
            self.cassava_leaf_disease = self.cassava_leaf_disease.iloc[idx_list, :]


    def __len__(self):
        return len(self.cassava_leaf_disease)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.cassava_leaf_disease.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform != None:
            image = self.transform(image)
        
        label = np.array(self.cassava_leaf_disease.iloc[idx, 1])
        return (image, label)

## Transforms and Dataloader

In [9]:
transforms = transforms.Compose([
    transforms.RandomResizedCrop(300),
    transforms.ToTensor(),
    transforms.Normalize([0.4303133, 0.49675637, 0.3135656], 
                         [0.2379062, 0.24065569, 0.22874062])
])

In [ ]:
class PL_Resnet18(pl.LightningModule):
    def __init__(self, criterion, optimizer, init_lr):
        super(PL_Resnet18, self).__init__()
        backbone_model = resnet18(pretrained=True)
        backbone_model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 128), nn.ReLU(), 
                                 nn.Linear(128, output_features), nn.LogSoftmax(dim=1)
                                )
        self.model = backbone_model
        self.criterion = criterion
        self.optimizer = optimizer
        self.init_lr = init_lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        log_ps = self.forward(x)
        train_loss = self.criterion(log_ps, y)
        tensorboard_logs = {'train_loss': train_loss}
        return {'loss': train_loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        log_ps = self.forward(x)
        val_loss = self.criterion(log_ps, y)
        return {'val_loss': val_loss}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}
    
    """
    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs}
    """
    
    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        optimizer = self.optimizer(self.model.fc.parameters(), lr=self.init_lr)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr= model_cfg["init_lr"],
                                                  total_steps=model_cfg["train_epochs"] * len(self.train_loader),
                                                  pct_start=0.4, div_factor=10)
        return [optimizer], [scheduler] 
    
    def prepare_data(self):
        cassava_dataset = CassavaDataset(csv_file=path_cfg['train_csv_path'], root_dir=path_cfg['train_img_path'], 
                                 transform=transforms)
        # split to train and validation sets
        cv_splits = get_cv_splits(path_cfg['train_csv_path'], cv_splits=5)
        
        self.train_data = Subset(cassava_dataset, cv_splits['split1_train'])
        self.val_data = Subset(cassava_dataset, cv_splits['split1_test'])

    def train_dataloader(self):
        self.train_loader = DataLoader(self.train_data, batch_size=train_cfg['batch_size'],shuffle=train_cfg['shuffle'],
                           num_workers=train_cfg["num_workers"])
        return self.train_loader

    def val_dataloader(self):
        self.val_loader = DataLoader(self.val_data, batch_size=valid_cfg['batch_size'],shuffle=valid_cfg['shuffle'],
                           num_workers=valid_cfg["num_workers"])
        return self.val_loader

    """
    def test_dataloader(self):
        loader = DataLoader(self.mnist_test, batch_size=64, num_workers=4)
        return loader
    """

In [ ]:
# loss function
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=model_cfg['init_lr'])

In [ ]:
scheduler = optim.lr_scheduler.OneCycleLR(

In [ ]:
trainer = pl.Trainer(gpus=1) # max_epochs=3, check_val_every_n_epoch=1

In [9]:
cassava_dataset = CassavaDataset(csv_file=path_cfg['train_csv_path'], root_dir=path_cfg['train_img_path'], 
                                 transform=transforms)

# split to train and validation sets
cv_splits = get_cv_splits(path_cfg['train_csv_path'], cv_splits=5)

# Datasets
train_data = Subset(cassava_dataset, cv_splits['split1_train'])
test_data  = Subset(cassava_dataset, cv_splits['split1_test'])

print('Len of train dataset = ', len(train_data))
print('Len of test dataset = ', len(test_data))

# Dataloaders
trainloader = DataLoader(train_data, batch_size=train_cfg['batch_size'],shuffle=train_cfg['shuffle'])
validateloader = DataLoader(test_data, batch_size=valid_cfg['batch_size'],shuffle=valid_cfg['shuffle'])

Len of train dataset =  17117
Len of test dataset =  4280


## Pretrained model

In [10]:
output_features = 5
model = models.resnet18(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 128), nn.ReLU(), 
                                 nn.Linear(128, output_features), nn.LogSoftmax(dim=1)
                                )
print('Trainable Parameters :', find_no_of_trainable_params(model))
#print(model.model.classifier)

Trainable Parameters : 66309


## Device, loss fn, optimizer

In [11]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device);



## Lr_find function

In [14]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1);
    fig.add_trace(trace1, row=1, col=2);
    #iplot(fig, show_link=False)
    fig.write_html(model_cfg['model_name'] + '_lr_find.html');

In [15]:
def find_lr(data_loader, init_value = 1e-8, final_value=100.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))
    
    for idx in progress_bar:
        batch_num += 1
        try:
            inputs, labels = next(dataloader_it)
        except StopIteration:
            dataloader_it = iter(data_loader)
            inputs, labels = next(dataloader_it)

        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        
        # handle exception in criterion
        try:
            # Forward pass
            log_ps = model(inputs)
            loss = criterion(log_ps, labels)
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results        
        
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss: {loss.item()},smoothed_loss: {smoothed_loss},lr : {lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(smooth_losses)
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
    return lr_finder_results

In [16]:
if path_cfg['lr_find'] == True:
    lr_finder_results = find_lr(trainloader)
    plot_lr_finder_results(lr_finder_results)

## Training & validation loops

In [17]:
# load previous weight file
if model_cfg['weight_path'] != '':
    state_dict = torch.load(model_cfg['weight_path'])
    model.load_state_dict(state_dict)

In [18]:
def validate(validate_dataloader):
    valid_it = iter(validate_dataloader)
    progress_bar = tqdm(range(len(validate_dataloader)))
    
    test_loss = 0
    test_accuracy = 0
    model.eval()

    with torch.no_grad():
        for batch_idx in progress_bar: 
            try:
                inputs, labels = next(valid_it)
            except StopIteration:
                valid_it = iter(validate_dataloader)
                inputs, labels = next(valid_it)

            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    test_loss = test_loss/len(validate_dataloader)
    test_accuracy = test_accuracy/len(validate_dataloader)
    return test_accuracy, test_loss

In [20]:
if path_cfg['train'] == True:
    results = {}
    results['train_losses'] = []
    #results['train_accuracy'] = []
    results['validate_losses'] = []
    results['validate_accuracy'] = []
    lr_list = []

    for epoch in range(model_cfg['train_epochs']):
        tr_it = iter(trainloader)
        progress_bar = tqdm(range(len(trainloader)))            
        running_loss = 0.0
        model.train()
        
        for batch_idx in progress_bar:
            try:
                inputs, labels = next(tr_it)
            except StopIteration:
                tr_it = iter(trainloader)
                inputs, labels = next(tr_it)

            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            log_ps = model(inputs)
            loss = criterion(log_ps, labels)

            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # lr scheduler
            scheduler.step()
            lr_list.append(optimizer.param_groups[0]['lr'])

            # store losses
            running_loss += loss.item()

            # print to console
            progress_bar.set_description(f"loss: {loss.item()} loss(avg): {running_loss/(batch_idx+1)}")
        
        results['train_losses'].append(running_loss/len(trainloader))
        
        # save weights periodically
        if (epoch % train_cfg['checkpt_every'] == 0):
            torch.save(model.state_dict(), model_cfg['model_name'] + str(epoch+1) + '_epochs.pth')
        
        # validate periodically
        if (epoch % valid_cfg['validate_every'] == 0):
            val_loss, val_accuracy = validate(validateloader)
            results['validate_losses'].append(val_loss)
            results['validate_accuracy'].append(val_accuracy)
    
    print('train_losses:', results['train_losses'])
    print('validate_losses:', results['validate_losses'])
    print('validate_accuracy:', results['validate_accuracy'])

loss: 1.102390170097351 loss(avg): 0.9727651359600441: 100%|██████████| 1070/1070 [10:04<00:00,  1.77it/s] 
100%|██████████| 268/268 [02:27<00:00,  1.82it/s]
loss: 0.7382745742797852 loss(avg): 0.7840901069273458: 100%|██████████| 1070/1070 [09:58<00:00,  1.79it/s] 
100%|██████████| 268/268 [02:25<00:00,  1.84it/s]
loss: 0.7612194418907166 loss(avg): 0.7268509004717675: 100%|██████████| 1070/1070 [10:00<00:00,  1.78it/s] 
100%|██████████| 268/268 [02:25<00:00,  1.84it/s]

train_losses: [0.9727651359600441, 0.7840901069273458, 0.7268509004717675]
validate_losses: [0.6919309701492538, 0.7325093283582089, 0.7516324626865671]
validate_accuracy: [0.7911192013590194, 0.7137122319927857, 0.6822951678464662]
